In [11]:
# Importing necessary libraries
from gensim.models import TfidfModel
import numpy as np
import json
import glob

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# #spacy
import spacy
from nltk.corpus import stopwords

# visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [12]:
#helper functions for data writing/reading
def load_data(file):
    # Function to load data from a JSON file
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data


def write_data(file, data):
    # Function to write data to a JSON file
    with open(file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [13]:
#helper functions for lda preprocessing

stopwords = stopwords.words("english") + ["go", "re", "just", "get", "want", "know", "so", "need", "knock", "look", "guy", "work", "say", "let", "come", "here", "make", "see",  "tell", "thing", "talk", "ve", "really"]


def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    # Function for lemmatizing the text and removing stopwords
    """https://spacy.io/api/annotation"""
    nlp = spacy.load(
        "/Users/anishpalakurthi/opt/anaconda3/lib/python3.8/site-packages/en_core_web_sm/en_core_web_sm-3.6.0", disable=["parser", "ner"])
    texts_out = []
    for sent in texts:
        # Lemmatizes each word by appending allowed tokens from the doc object's metadata
        doc = nlp(sent)
        texts_out.append(" ".join([token.lemma_ for token in doc if token.pos_ in allowed_postags]))

    # Removing stopwords from the lemmatized texts
    for word in texts_out:
        if word in stopwords:
            texts_out.remove(word)
    return texts_out


def gen_words(texts):
    # Function to preprocess the lemmatized keywords
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return final

In [14]:
#handle bigrams and trigrams
def make_bigrams(texts):
    # Applying bigram models to the data_words
    bigram_phrases = gensim.models.Phrases(texts, min_count=5, threshold=50)
    bigram = gensim.models.phrases.Phraser(bigram_phrases)
    return [bigram[doc] for doc in texts]


def make_trigrams(texts):
    # Applying trigram models to the data_words
    bigram_phrases = gensim.models.Phrases(texts, min_count=5, threshold=50)
    trigram_phrases = gensim.models.Phrases(bigram_phrases[texts], threshold=50)
    bigram = gensim.models.phrases.Phraser(bigram_phrases)
    trigram = gensim.models.phrases.Phraser(trigram_phrases)
    return [trigram[bigram[doc]] for doc in texts]


In [15]:
# Load and preprocess data with helper calls
episodeData = load_data("cleaned.json")
plots = [episodeData[episode] for episode in episodeData]
lemmatized_texts = lemmatization(plots)
data_words = gen_words(lemmatized_texts)


In [16]:
# Create bigrams and trigrams
data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_words)

In [17]:
# Applying TF-IDF removal to the corpus
id2word = corpora.Dictionary(data_bigrams_trigrams)
texts = data_bigrams_trigrams
corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

In [18]:
low_value = 0.03
words = []
words_missing_in_tfidf = []


# Identifying words to be removed based on their TF-IDF values
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]

    drops = low_value_words + words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])

    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [19]:
# Reconstructing the corpus after TF-IDF removal
id2word = corpora.Dictionary(data_words)
corpus = [id2word.doc2bow(text) for text in data_words]


In [20]:
# Generating the LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=5,
                                            random_state=100, update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)

# Enabling the notebook for visualization
pyLDAvis.enable_notebook()

# Preparing the data for visualization using pyLDAvis
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)

# Displaying the visualization
vis

/Users/anishpalakurthi/Library/Python/3.10/lib/python/site-packages/sklearn/manifold/_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.355518  0.033671       1        1  82.797281
3     -0.230016  0.117614       2        1   8.626511
7     -0.195465 -0.133612       3        1   5.141235
10    -0.082203  0.230891       4        1   3.316889
6      0.119015 -0.033527       5        1   0.057219
11     0.106045 -0.037792       6        1   0.028944
8      0.109523 -0.025794       7        1   0.025157
1      0.105724 -0.030290       8        1   0.001381
2      0.105724 -0.030290       9        1   0.001379
0      0.105725 -0.030291      10        1   0.001347
5      0.105725 -0.030290      11        1   0.001336
9      0.105721 -0.030289      12        1   0.001322, topic_info=       Term         Freq        Total Category  logprob  loglift
210      go  4713.000000  4713.000000  Default  30.0000  30.0000
417      re  4586.000000  4586.000000  Default  29.0000  29.0000
207     get  2925.000000  2925.000000  Default  28.0000  28.0000
529    time  1380.000000  1380.000000  Default  27.0000  27.0000
234    here  1448.000000  1448.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
322    mean     0.000229  1073.633060  Topic12  -9.4924  -4.1258
90     come     0.000229  1203.458658  Topic12  -9.4924  -4.2400
302  little     0.000229  1069.324760  Topic12  -9.4924  -4.1218
295     let     0.000229   974.563806  Topic12  -9.4925  -4.0290
567     way     0.000229   754.067082  Topic12  -9.4925  -3.7726

[860 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
7242      3  0.871130  acquaintance
2334      1  0.360491     agreement
2334      2  0.009487     agreement
2334      3  0.066406     agreement
2334      4  0.559709     agreement
...     ...       ...           ...
580       4  0.003290          work
8429      1  0.667169          worn
583       1  0.980927         worry
583       2  0.012907         worry
583       3  0.004302         worry

[652 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 8, 11, 7, 12, 9, 2, 3, 1, 6, 10])